In [1]:
import pandas as pd
import numpy as np
import re
import os
import re
import emoji
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

In [3]:
# Competition metric
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [8]:
display(df.head())
print(df['sentiment'].unique())
print(df.shape)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


['neutral' 'negative' 'positive']
(27481, 4)


## Additional task*
___________
Lets predict sentiment for these sentences

In [188]:
def text_preprocessing(text):
    
    # lower case
    text = text.lower()

In [228]:
#  Идеи: 
# заменить все hhtps на определённые символы
# 
# 
# 
# 
# 